# POC PYTHON #
- ## Pyspark

In [1]:
pip install pyspark


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=cae6719b04eb9273320977d9078f05a1a66e7c2e6dfc4c7f7edc52de5f5782cb
  Stored in directory: c:\users\charles.lavignasse\appdata\local\pip\cache\wheels\32\97\d3\8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark


In [ ]:
import pyspark as pk
import pandas as pd
#https://runawayhorse001.github.io/LearningApacheSpark/pyspark.pdf
print(pk.__version__)

3.2.0


In [31]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
sqlsUrl = config['SQLSERVER']['host']
username = config['SQLSERVER']['username']
password = config['SQLSERVER']['password']
port = config['SQLSERVER']['port']


In [ ]:
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')
database = config['TEST']['database']
sqlsUrl = config['TEST']['host']
username = config['TEST']['username']
password = config['TEST']['password']
port = config['TEST']['port']



In [6]:
# jdbc_url = jdbc:sqlserver://{config.get('mydb', 'host')}:{config.get('mydb', 'port')};database={config.get('mydb', 'database')}"
# connection_properties = {
#     "user": config.get('mydb', 'username'),
#     "password": config.get('mydb', 'password')
# }

In [11]:
# from pyspark.sql import SparkSession

# jars = [
#     "mssql-jdbc-6.4.0.jre8.jar",
# ]
# spark = (SparkSession
#   .builder
#   .appName("PySpark with SQL server")
#   .config("spark.driver.extraClassPath", ":".join(jars))
#   .getOrCreate())  


# .option("driver", "com.microsoft.azure:spark-mssql-connector_2.12:1.2.0")\

#Problème de version JAVA avec pyspark, peut être problématique dans le cadre de BABILOU qui utilise java pour Talend
#Résolu en téléchargeant JDK8
#https://stackoverflow.com/questions/31841509/pyspark-exception-java-gateway-process-exited-before-sending-the-driver-its-po


In [17]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName('Connection-Test')\
    .getOrCreate()

In [21]:


qryStr = """ (
SELECT * FROM STG_PROCARE_AR_SCHEDULE
    ) t """

# spark.read.format('jdbc')\
#     .option('url',sqlsUrl)\
#     .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
#     .option('dbtable', qryStr )\
#     .option("user", user) \
#     .option("password", passwd) \
#     .load().show()

# jdbcDF = spark.read \
#     .format("com.microsoft.sqlserver.jdbc.spark") \
#     .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
#     .option("authentication", "ActiveDirectoryPassword") \
#     .option("encrypt", "true") \
#     .option("hostNameInCertificate", "*.database.windows.net") \
#     .option("url", sqlsUrl+":"+port) \
#     .option("dbtable", qryStr) \
#     .option("user", username) \
#     .option("password", password) \
#     .load()

jdbc_df = spark.read.format("jdbc") \
       .option("url", sqlsUrl+port) \
       .option("driver", "SQL Server Native Client 11.0")\
       .option("dbtable", qryStr) \
       .option("authentication", "ActiveDirectoryPassword") \
       .option("user", username) \
       .option("password", password) \
       .option("encrypt", "true") \
       .option("databaseName", "STG_BABILOU") \
       .option("hostNameInCertificate", "*.database.windows.net") \
       .load()


Py4JJavaError: An error occurred while calling o202.load.
: java.lang.ClassNotFoundException: SQL Server Native Client 11.0
	at java.net.URLClassLoader.findClass(URLClassLoader.java:387)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:101)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:101)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:39)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:33)
	at org

In [2]:
jdbc_df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("authentication", "ActiveDirectoryPassword") \
        .option("user", user_name) \
        .option("password", password) \
        .option("encrypt", "true") \
        .option("hostNameInCertificate", "*.database.windows.net") \
        .load()

NameError: name 'spark' is not defined

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession\
    .builder\
    .appName('Read_csv')\
    .getOrCreate()

In [3]:
procare = spark.read.options(header=True)\
.options(delimiter=';')\
.options(inferschema='true')\
.csv("C:\\Users\\charles.lavignasse\\Desktop\\POC-PYTHON\\notebooks\\Connection\procare.csv")

etp= spark.read.options(header=True)\
.options(delimiter=';')\
.options(inferschema='true')\
.csv("C:\\Users\\charles.lavignasse\\Desktop\\POC-PYTHON\\notebooks\\Connection\etp.csv")

In [4]:
procare.show()

+-----------+
|HoursWorked|
+-----------+
|        4.5|
|       15.5|
|       1.55|
|      13.75|
|   9.983333|
|  10.016666|
|  10.383333|
|  11.833333|
|        6.5|
|       8.75|
|       7.75|
|   2.166666|
|       1.75|
|   9.333333|
|  10.666666|
|   9.083333|
|       14.5|
|   8.983333|
|       17.5|
|  10.583333|
+-----------+
only showing top 20 rows



In [10]:
etp.collect()[0][0] 

TypeError: 'Row' object does not support item assignment

In [26]:

if len(etp.collect())==0: 
    print('empty')


empty


In [29]:
etp['MinDailyHour'][1]

TypeError: 'Column' object is not callable

In [3]:
spark

In [ ]:
for ligne in procare : 
    CellValue =[]
    for ninter in etp :
        if etp['MinDailyHour'][ninter] <= procare['HoursWorked'][ligne] <= etp['MaxDailyHour'][ninter] Then pass
    CellValue =[etp['Correspondance ETP'][ninter]]
    procare['ETP'][ligne]=CellValue
